In [ ]:
#######################################################################################
# Please be aware that running this code may yield results different from those       #
# reported in my thesis. This difference is due to stochastic weight initialization   #
# in LSTMs. Results can therfore vary with each run.                                  #
#######################################################################################

In [ ]:
# Install packages if needed
#pip install yfinance
#pip install numpy
#pip install pandas
#pip install scikit-learn
#pip install tensorflow
#pip install matplotlib
#pip install seaborn

In [3]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

In [4]:
# Function for creating sequential input data
def create_lstm_data(data_set_scaled,num_features ,time_steps=1):
    X = []

    for j in range(num_features):
        X.append([])
        for i in range(time_steps, data_set_scaled.shape[0]):
            X[j].append(data_set_scaled[i-time_steps:i, j])

    X=np.moveaxis(X, [0], [2])

    X, yi =np.array(X), np.array(data_set_scaled[time_steps:,0])
    y=np.reshape(yi,(len(yi),1))

    return X, y

In [7]:
class ANN:
    def __init__(self,ticker,forecasting_window, batch_size = 16 ,time_steps = 10,start_date = "2009-01-01",end_date = "2019-02-01"):

        #Data Collection
        self.ticker = ticker
        self.start_date = start_date
        self.end_date = end_date

        # Data Preparation
        self.scaler = StandardScaler() 
        self.daily_returns_scaled = None
        self.daily_prices = None
        self.daily_returns = None
        self.daily_volume = None
        self.daily_returns_sqr = None
        self.daily_volume_scaled = None
        self.combined_features = None
        ###################
        self.time_steps = time_steps
        ###################
        self.train_size = None
        self.val_size = None
        self.test_size = None
        self.x_returns = None
        self.x_volume = None
        self.x = None
        self.y = None
        self.x_train = None
        self.x_test = None
        self.x_val = None
        self.y_val = None
        self.y_train = None
        self.y_test = None
        self.feature_count = None
        self.window = forecasting_window
        
        # Model Training
        self.confidence = 0.9
        self.model = None
        self.epochs = 100
        self.batch_size = batch_size
        self.patience = 5
        self.sym_loss = [lambda y,f: self.q_loss((1-self.confidence)/2,y,f), lambda y,f: self.q_loss(self.confidence +((1-self.confidence)/2) ,y,f)]


        # Model Testing
        self.combined_predictions = None
        self.all_predictions = None
        self.number_of_steps = None
        self.y_test_inv = None
        self.pred_inv = None
        self. eta = 30 #Weighting factor for penalization in the CWC
        
        # Graphs 
        self.daily_dates = None
        self.dates_test = None

        
    # Define quantile loss function
    def q_loss(self, q, y, f):
        e = (y - f)
        return K.mean(K.maximum(q * e, (q - 1) * e), axis=-1)

    # Function for downloading the data if only the daily returns are used as a feature:
    def read_data_return(self):
        # Get data from Yahoo Finance
        data = yf.download(self.ticker, start=self.start_date, end=self.end_date)

        # Extract closing prices 
        self.daily_prices = data["Close"]

        # Calculate the daily return of the stock
        self.daily_returns = (self.daily_prices / self.daily_prices.shift(1)) - 1
        self.daily_returns = self.daily_returns.dropna()

        df = pd.DataFrame({
            "Returns": self.daily_returns
        })
        self.feature_count = len(df.axes[1])

        # Scale the data
        scaled_df = self.scaler.fit_transform(df)
        
        # Create sequential data for the QLSTM
        self.x, self.y = create_lstm_data(data_set_scaled = scaled_df,time_steps = self.time_steps, num_features = self.feature_count)

        # Split the data into training, validation, and test sets
        total_size = len(self.x)
        self.train_size = int(total_size * 0.6)
        self.val_size = int(total_size * 0.2)
        self.test_size = total_size - self.train_size - self.val_size 

    # Function for downloading the data if the daily returns and the daily trading volume are used as features:
    def read_data_return_vol(self):
        # Get data from Yahoo Finance
        data = yf.download(self.ticker, start=self.start_date, end=self.end_date)
        # Extract closing prices and volume
        self.daily_prices = data["Close"]
        self.volume = data["Volume"]

        # Calculate the daily return of the stock
        self.daily_returns = (self.daily_prices / self.daily_prices.shift(1)) - 1
        self.daily_returns = self.daily_returns.dropna()

        # Align volume data with the daily_returns
        self.volume = self.volume[self.daily_returns.index]
        
        self.daily_returns_sqr = self.daily_returns**2

        df = pd.DataFrame({
            "Returns": self.daily_returns,
            "Volume": self.volume[self.daily_returns.index]  # Gleiche Indizes
        })
        self.feature_count = len(df.axes[1])

        # Scale the data
        scaled_df = self.scaler.fit_transform(df)
        
        # Create sequential data for the QLSTM
        self.x, self.y = create_lstm_data(data_set_scaled = scaled_df,time_steps = self.time_steps, num_features = self.feature_count)

        # Split the data into training, validation, and test sets
        total_size = len(self.x)
        self.train_size = int(total_size * 0.6)
        self.val_size = int(total_size * 0.2)
        self.test_size = total_size - self.train_size - self.val_size 

    # The three model configurations that are tested:
    def lstm_model_complex(self, input_shape):
        # Input layer
        inputs = Input(shape=input_shape) 
        # Three LSTM layers with 100 neurons each and a dropout rate of 0.1
        lstm1 = LSTM(units=100, return_sequences=True,dropout= 0.1)(inputs)
        lstm2 = LSTM(units=100, return_sequences=True,dropout= 0.1)(lstm1)
        lstm3 = LSTM(units=100,dropout= 0.1)(lstm2)

        # Output layers for quantiles
        out10 = Dense(1)(lstm3)
        out90 = Dense(1)(lstm3)

        model = Model(inputs, [out10, out90]) 
        return model


    def lstm_model_mid(self, input_shape):
        # Input layer
        inputs = Input(shape=input_shape)
        # Two LSTM layers with 80 neurons each and a dropout rate of 0.1
        lstm1 = LSTM(units=80, return_sequences=True,dropout= 0.1)(inputs)
        lstm2 = LSTM(units=80,dropout= 0.1)(lstm1)

        # Output layers for quantiles
        out10 = Dense(1)(lstm2)
        out90 = Dense(1)(lstm2)

        model = Model(inputs, [out10, out90])
        return model
    
    def lstm_model_simpler(self, input_shape):
        # Input layer
        inputs = Input(shape=input_shape)
        # One LSTM layers with 25 neurons and a dropout rate of 0.1
        lstm1 = LSTM(units=25,dropout= 0.1)(inputs)

        # Output layers for quantiles
        out10 = Dense(1)(lstm1)
        out90 = Dense(1)(lstm1)

        model = Model(inputs, [out10, out90])
        return model

    # Function for updating the training data when the forecasting window is shifted
    def update_data_splits(self, i):
        roll = i * self.window

        self.x_train, self.y_train = self.x[:self.train_size+roll], self.y[:self.train_size+roll]
        self.x_val, self.y_val = self.x[self.train_size+roll:self.train_size + self.val_size+roll], self.y[self.train_size+roll:self.train_size + self.val_size+roll]
        self.x_test, self.y_test = self.x[self.train_size + self.val_size+roll:self.train_size + self.val_size+roll+self.window], self.y[self.train_size + self.val_size +roll:self.train_size + self.val_size+roll+self.window]

        if not isinstance(self.y_train, list) or len(self.y_train) != 2:
            self.y_train = [self.y_train, self.y_train] 
        if not isinstance(self.y_test, list) or len(self.y_test) != 2:
            self.y_test = [self.y_test, self.y_test] 

    # Forecasting function if only the daily returns are used as a feature
    def rolling_forecast(self, loss, model):
        # Read and prepare data
        self.read_data_return()

        input_shape = (self.time_steps, self.feature_count)
        self.model = model(input_shape) # The medium model is used
        
        # Compile model with quantile losses
        losses = loss
        self.model.compile(loss=losses, optimizer="adam", loss_weights=[0.5, 0.5])

        stop_early = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5) # early stopping 
        self.number_of_steps = self.test_size // self.window

        # Initialize container for combined predictions
        self.combined_predictions = [np.empty((0, 1)), np.empty((0, 1))]

        for step in range(self.number_of_steps):
            # Update the data splits for this window
            self.update_data_splits(step)

            # Train the model
            self.model.fit(self.x_train, self.y_train, 
                           validation_data=(self.x_val, self.y_val), 
                           epochs=self.epochs, batch_size=self.batch_size, 
                           callbacks=[stop_early])

            # Make predictions
            predictions = self.model.predict(self.x_test)

            # Add current predictions to the combined predictions
            self.combined_predictions[0] = np.vstack([self.combined_predictions[0], predictions[0]])
            self.combined_predictions[1] = np.vstack([self.combined_predictions[1], predictions[1]])
    
     # Forecasting function if the daily returns and the daily trading volume are used as features
    def rolling_forecast_v(self, loss,model):
        # Read and prepare data
        self.read_data_return_vol()

        input_shape = (self.time_steps, self.feature_count)
        self.model = model(input_shape) # The medium model is used
        
        # Compile model with quantile losses
        losses = loss
        self.model.compile(loss=losses, optimizer="adam", loss_weights=[0.5, 0.5])

        stop_early = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5) # early stopping 
        self.number_of_steps = self.test_size // self.window
        # Initialize container for combined predictions
        self.combined_predictions = [np.empty((0, 1)), np.empty((0, 1))]

        for step in range(self.number_of_steps):
            # Update the data splits for this window
            self.update_data_splits(step)

            # Train the model
            self.model.fit(self.x_train, self.y_train, 
                           validation_data=(self.x_val, self.y_val), 
                           epochs=self.epochs, batch_size=self.batch_size, 
                           callbacks=[stop_early])

            # Make predictions
            predictions = self.model.predict(self.x_test)

            # Add current predictions to the combined predictions
            self.combined_predictions[0] = np.vstack([self.combined_predictions[0], predictions[0]])
            self.combined_predictions[1] = np.vstack([self.combined_predictions[1], predictions[1]])

    # Test the QLSTM on the test set
    def predict_eval(self):
        self.y_test = self.y[self.train_size + self.val_size :self.train_size + self.val_size+self.number_of_steps*self.window]
        
        # Calculate upper and lower bound of the prediction intervals
        lower_bound = self.combined_predictions[0]  
        upper_bound = self.combined_predictions[1]  

        # Calculate the prediction interval normalized average width (PINAW)
        n = len(self.y_test)
        interval_width_sum = np.sum(upper_bound - lower_bound)
        y_range = np.max(self.y_test) - np.min(self.y_test)
        pinaw = interval_width_sum / (n * y_range)
        
        # Calculate the prediction interval coverage probability (PICP)
        within_interval = np.sum((self.y_test >= lower_bound) & (self.y_test <= upper_bound))
        picp = within_interval / n

        # Calculate Coverage Width-based Criterion (CWC)
            # Eta is the weighting factor for penalization
        cwc = (1 - pinaw) * np.exp(-self.eta * (picp - (self.confidence))**2)
        
        return {
            "batch size": self.batch_size,
            "time steps": self.time_steps,
            "ticker": self.ticker,
            "interval_width_sum":interval_width_sum,
            "within_interval":within_interval,
            "LSTM CWC": cwc,
            "LSTM PINAW": pinaw,
            "LSTM PICP": picp,
        }
 
    # Function for plotting the results 
    def graph(self):

        self.pred_inv = [
        self.scaler.inverse_transform(self.combined_predictions[0].reshape(-1, 1)),
        self.scaler.inverse_transform(self.combined_predictions[1].reshape(-1, 1))
        ]
       
        self.y_test_inv = self.daily_returns[self.train_size + self.val_size:self.train_size + self.val_size + self.number_of_steps*self.window]
        
        self.daily_dates = self.daily_returns.index[self.train_size + self.val_size:self.train_size + self.val_size + self.number_of_steps*self.window]
        

        plt.figure(figsize=(14, 5))
        plt.plot(self.daily_dates, self.y_test_inv, label="Actual Daily Return ", color="black")
        plt.plot(self.daily_dates, self.pred_inv[0], label=f"Predicted Return (Quantile {round(((1-self.confidence)/2)*100,0)}%)", linestyle="dotted", color = "purple")
        plt.plot(self.daily_dates, self.pred_inv[1], label=f"Predicted Return (Quantile {(self.confidence +((1-self.confidence)/2))*100}%)", linestyle="dotted", color = "green")

        y1 = self.pred_inv[0].ravel()  
        y2 = self.pred_inv[1].ravel()  

        plt.fill_between(self.daily_dates, y1, y2, where=(y2 >= y1), color="red", alpha=0.3)

        plt.xlabel("Date")
        plt.ylabel("Daily Return")
        plt.legend(loc="upper right") 
        plt.show()
        

In [ ]:
# Test with the stock of Boeing:
QLSTM_test = ANN("BA",start_date="2012-05-03",end_date= "2015-09-01",time_steps =15, batch_size = 32,forecasting_window = 10)
QLSTM_test.rolling_forecast(loss= QLSTM_test.sym_loss,model = QLSTM_test.lstm_model_mid)
QLSTM_test.predict_eval()
QLSTM_test.graph()

In [ ]:
# Determining the start of the test set for each period so it can be excluded from the hyperparameter tuning:

# The start and end dates of the three study periods
start_dates = ["2009-01-01", "2012-05-02", "2015-09-01"]
end_dates = ["2012-05-02", "2015-09-01", "2019-01-01"]

# Loop through each pair of start and end dates
for i,(start, end) in enumerate(zip(start_dates, end_dates)):

    # Convert the start and end dates to datetime format
    start_date = pd.to_datetime(start)
    end_date = pd.to_datetime(end)
    
    # Calculate the total duration between the start and end date
    total_duration = end_date - start_date
    
    # The test period will be the last 20% of the total period
    test_duration = total_duration * 0.2
    
    # The start of the test period will be at 80% of the total period
    test_date = end_date - test_duration
    
    # Print the start of the test period
    print(f"Start of test period {i+1}: {test_date.date()}")


In [ ]:
# Code used for testing the three different QLSTM architectures:

time_steps = [5, 10, 15, 20, 25, 30]
batch_size = [16, 32, 64, 128]

# United Parcel Service for the first study period
results_ups_simp = []
for rep in range(1):
    for time in time_steps:
        for batch in batch_size:
                    QLSTM_opt = ANN("UPS",start_date="2009-01-01",end_date= "2011-09-01",time_steps =time, batch_size = batch,forecasting_window = 10)
                    QLSTM_opt.rolling_forecast(loss= QLSTM_opt.sym_loss, model= QLSTM_opt.lstm_model_simpler)
                    result = QLSTM_opt.predict_eval()
                    results_ups_simp.append({
                        "time steps": result["time steps"],
                        "batch size": result["batch size"],
                        "CWC": result["LSTM CWC"]
                    })

df_results_ups_simp = pd.DataFrame(results_ups_simp)
#df_results_ups_simp.to_csv("ups_simple.csv", index=False)
print(df_results_ups_simp)


results_ups_mid = []
for rep in range(9):
    for time in time_steps:
        for batch in batch_size:
                    QLSTM_opt = ANN("UPS",start_date="2009-01-01",end_date= "2011-09-01",time_steps =time, batch_size = batch,forecasting_window = 10)
                    QLSTM_opt.rolling_forecast(loss= QLSTM_opt.sym_loss, model= QLSTM_opt.lstm_model_mid)
                    result = QLSTM_opt.predict_eval()
                    results_ups_mid.append({
                        "time steps": result["time steps"],
                        "batch size": result["batch size"],
                        "CWC": result["LSTM CWC"]
                    })

df_results_ups_mid = pd.DataFrame(results_ups_mid)
#df_results_ups_mid.to_csv("ups_medium.csv", index=False)
print(df_results_ups_mid)


results_ups_comp = []
for rep in range(9):
    for time in time_steps:
        for batch in batch_size:
                    QLSTM_opt = ANN("UPS",start_date="2009-01-01",end_date= "2011-09-01",time_steps =time, batch_size = batch,forecasting_window = 10)
                    QLSTM_opt.rolling_forecast(loss= QLSTM_opt.sym_loss, model= QLSTM_opt.lstm_model_complex)
                    result = QLSTM_opt.predict_eval()
                    results_ups_comp.append({
                        "time steps": result["time steps"],
                        "batch size": result["batch size"],
                        "CWC": result["LSTM CWC"]
                    })

df_results_ups_comp = pd.DataFrame(results_ups_comp)
#df_results_ups_comp.to_csv("ups_complex.csv", index=False)
print(df_results_ups_comp)

# Waste Management for the second study period
results_wm_simp = []
for rep in range(9):
    for time in time_steps:
        for batch in batch_size:
                    QLSTM_opt = ANN("WM",start_date="2012-05-02",end_date= "2014-12-31",time_steps =time, batch_size = batch,forecasting_window = 10)
                    QLSTM_opt.rolling_forecast(loss= QLSTM_opt.sym_loss, model= QLSTM_opt.lstm_model_simpler)
                    result = QLSTM_opt.predict_eval()
                    results_ups_simp.append({
                        "time steps": result["time steps"],
                        "batch size": result["batch size"],
                        "CWC": result["LSTM CWC"]
                    })

df_results_wm_simp = pd.DataFrame(results_wm_simp)
#df_results_wm_simp.to_csv("wm_simple.csv", index=False)
print(df_results_wm_simp)

results_wm_mid = []
for rep in range(9):
    for time in time_steps:
        for batch in batch_size:
                    QLSTM_opt = ANN("WM",start_date="2012-05-02",end_date= "2014-12-31",time_steps =time, batch_size = batch,forecasting_window = 10)
                    QLSTM_opt.rolling_forecast(loss= QLSTM_opt.sym_loss, model= QLSTM_opt.lstm_model_mid)
                    result = QLSTM_opt.predict_eval()
                    results_ups_mid.append({
                       "time steps": result["time steps"],
                        "batch size": result["batch size"],
                        "CWC": result["LSTM CWC"]
                    })

df_results_wm_mid = pd.DataFrame(results_wm_mid)
#df_results_wm_mid.to_csv("wm_medium.csv", index=False)
print(df_results_wm_mid)

results_wm_comp = []
for rep in range(9):
    for time in time_steps:
        for batch in batch_size:
                    QLSTM_opt = ANN("WM",start_date="2012-05-02",end_date= "2014-12-31",time_steps =time, batch_size = batch,forecasting_window = 10)
                    QLSTM_opt.rolling_forecast(loss= QLSTM_opt.sym_loss, model= QLSTM_opt.lstm_model_complex)
                    result = QLSTM_opt.predict_eval()
                    results_ups_comp.append({
                        "time steps": result["time steps"],
                        "batch size": result["batch size"],
                        "CWC": result["LSTM CWC"]
                    })

df_results_wm_comp = pd.DataFrame(results_wm_comp)
#df_results_wm_comp.to_csv("wm_complex.csv", index=False)
print(df_results_wm_comp)

In [ ]:
# Code used for testing different features:

# Start and end dates for the three study periods without their test sets
start_dates =["2009-01-01", "2012-05-02", "2015-09-01"]
end_dates = ["2011-09-01", "2014-12-31", "2018-05-02"]

industrials_stocks = ["BA", "CHRW", "DOV", "EFX", "EMR", "FAST", "ITW", "NOC", "UPS", "WM"]

results_return = []
for rep in range(9):
    for i in range(len(start_dates)):
        for stock in industrials_stocks:
                    QLSTM_opt = ANN(stock,start_date=start_dates[i],end_date= end_dates[i],time_steps =15, batch_size = 32,forecasting_window = 10)
                    QLSTM_opt.rolling_forecast(loss= QLSTM_opt.sym_loss, model = QLSTM_opt.lstm_model_mid)
                    result = QLSTM_opt.predict_eval()
                    results_return.append({
                        "interval_width_sum":result["interval_width_sum"],
                        "within_interval":result["within_interval"],
                        "PINAW": result["LSTM PINAW"],
                        "PICP": result["LSTM PICP"],
                        "CWC": result["LSTM CWC"],
                        "Stock": QLSTM_opt.ticker,
                        "Study Period": i + 1
                    })

    df_return = pd.DataFrame(results_return)
        
    #file_name = f"f_return_p{i+1}.csv"  
    #df_return.to_csv(file_name, index=False)
    print(df_return)



results_return_vol = []
for rep in range(9):
    for i in range(len(start_dates)):
        for stock in industrials_stocks:
                    QLSTM_opt = ANN(stock,start_date=start_dates[i],end_date= end_dates[i],time_steps =15, batch_size = 32,forecasting_window = 10)
                    QLSTM_opt.rolling_forecast_v(loss= QLSTM_opt.sym_loss, model = QLSTM_opt.lstm_model_mid)
                    result = QLSTM_opt.predict_eval()
                    results_return_vol.append({
                        "interval_width_sum":result["interval_width_sum"],
                        "within_interval":result["within_interval"],
                        "PINAW": result["LSTM PINAW"],
                        "PICP": result["LSTM PICP"],
                        "CWC": result["LSTM CWC"],
                        "Stock": QLSTM_opt.ticker,
                        "Study Period": i + 1 
                    })


    df_return_vol = pd.DataFrame(results_return_vol)
        
    #file_name = f"f_return_vol_p{i+1}.csv"  
    #df_return_vol.to_csv(file_name, index=False)
    print(df_return_vol)

In [ ]:
# Code used for the final testing of all stocks:

# Start and end dates of the three study periods
start_dates =["2009-01-01", "2012-05-02", "2015-09-01"]
end_dates = ["2012-05-02", "2015-09-01", "2019-01-01"]

industrials_stocks = ["BA", "CHRW", "DOV", "EFX", "EMR", "FAST", "ITW", "NOC", "UPS", "WM"]

results = []
for rep in range(9):
    for i in range(len(start_dates)):
        for stock in industrials_stocks:
                    QLSTM_opt = ANN(stock,start_date=start_dates[i],end_date= end_dates[i],time_steps =15, batch_size = 32,forecasting_window = 10)
                    QLSTM_opt.rolling_forecast_v(loss= QLSTM_opt.sym_loss, model = QLSTM_opt.lstm_model_mid)
                    result = QLSTM_opt.predict_eval()
                    results.append({
                        "interval_width_sum":result["interval_width_sum"],
                        "within_interval":result["within_interval"],
                        "LSTM PINAW": result["LSTM PINAW"],
                        "LSTM PICP": result["LSTM PICP"],
                        "CWC": result["LSTM CWC"],
                        "Stock": QLSTM_opt.ticker,
                        "Study Period": i + 1
                    })

df_results = pd.DataFrame(results)

#df_results.to_csv(final_results_QLSTM, index=False)
print(df_results)
        

In [ ]:
# Plotting the results for the three different QLSTM architectures:

stocks = ["wm", "ups"]

# Loop through each stock and generate the plot
for stock in stocks:
    # Read CSV files dynamically based on the stock
    df_simple = pd.read_csv(f"QLSTM_hyperparameter_results/{stock}_simple.csv")
    df_mid = pd.read_csv(f"QLSTM_hyperparameter_results/{stock}_medium.csv")
    df_complex = pd.read_csv(f"QLSTM_hyperparameter_results/{stock}_complex.csv")

    # Add a new column to identify the model type
    df_simple["model"] = "Simple"
    df_mid["model"] = "Medium"
    df_complex["model"] = "Complex"

    # Combine the DataFrames
    df_combined = pd.concat([df_simple, df_mid, df_complex], ignore_index=True)

    # Create a categorical column for "time steps"
    df_combined["time steps"] = df_combined["time steps"].astype("category")

    # Create the boxplot
    plt.figure(figsize=(12, 8))

    # Use Seaborn for the boxplot with color assignment based on "time steps"
    sns.boxplot(x="model", y="CWC", hue="time steps", data=df_combined, palette="mako", whis=(0, 100))

    # Set plot configurations
    plt.xlabel("Model",fontsize = 20)
    plt.ylabel("CWC", fontsize = 18)
    plt.xticks(rotation=45, fontsize=18)  
    plt.yticks(fontsize=14)  
    plt.legend(title="Time Steps", fontsize = 13)

    # Adjust layout and save the plot
    plt.tight_layout()
    #plt.savefig(f"boxplot_time_QLSTM_comparison_{stock}.png", dpi=300)
    plt.show()

In [ ]:
# Plotting the results for the three different QLSTM architectures:

stocks = ["wm", "ups"]

# Loop through each stock and generate the plot
for stock in stocks:
    # Read CSV files dynamically based on the stock
    df_simple = pd.read_csv(f"QLSTM_hyperparameter_results/{stock}_simple.csv")
    df_mid = pd.read_csv(f"QLSTM_hyperparameter_results/{stock}_medium.csv")
    df_complex = pd.read_csv(f"QLSTM_hyperparameter_results/{stock}_complex.csv")

    # Add a new column to identify the model type
    df_simple["model"] = "Simple"
    df_mid["model"] = "Medium"
    df_complex["model"] = "Complex"

    # Combine the DataFrames
    df_combined = pd.concat([df_simple, df_mid, df_complex], ignore_index=True)

    # Create a categorical column for "time steps"
    df_combined["batch size"] = df_combined["batch size"].astype("category")

    # Create the boxplot
    plt.figure(figsize=(12, 8))

    # Use Seaborn for the boxplot with color assignment based on "time steps"
    sns.boxplot(x="model", y="CWC", hue="batch size", data=df_combined, palette="viridis", whis=(0, 100))

    # Set plot configurations
    plt.xlabel("Model",fontsize = 20)
    plt.ylabel("CWC", fontsize = 18)
    plt.xticks(rotation=45, fontsize=18)  
    plt.yticks(fontsize=14)  
    plt.legend(title="Batch Size", fontsize = 13)

    # Adjust layout and save the plot
    plt.tight_layout()
    #plt.savefig(f"boxplot_time_QLSTM_comparison_{stock}.png", dpi=300)
    plt.show()

In [ ]:
# Plotting excerpt of the comparison of different features:

stocks = ["EFX", "EMR"]

# Read CSV files 
df_return = pd.read_csv("QLSTM_features_results/f_return_p1.csv")
df_return_vol = pd.read_csv("QLSTM_features_results/f_return_vol_p1.csv")

# Add model type column
df_return["model"] = "Daily Returns"
df_return_vol["model"] = "Volume and Daily Returns"

# Combine the data frames
df_combined = pd.concat([df_return, df_return_vol], ignore_index=True)


# Loop through stocks
for  stock in stocks:

    # Create the plot 
    plt.figure(figsize=(6, 4))

    # Filter for the current stock  
    df_filtered = df_combined[df_combined["Stock"] == stock]

    # Create boxplot
    sns.boxplot(x="model", y="CWC", hue="model", palette="mako", data=df_filtered,whis=(0, 100))
    plt.xlabel("Features")
    plt.ylabel("CWC")
    
    # Adjust layout and display the plot
    plt.tight_layout()
    plt.show()

In [ ]:
# Plotting the results for the comparison of different features:

study_periods = [1, 2, 3]
stocks = ["BA", "UPS", "WM", "DOV", "NOC", "EMR", "ITW", "CHRW", "FAST", "EFX"]

# Loop through all study periods
for period in study_periods:
    # Read CSV files for the current study period
    df_return = pd.read_csv(f"QLSTM_features_results/f_return_p{period}.csv")
    df_return_vol = pd.read_csv(f"QLSTM_features_results/f_return_vol_p{period}.csv")

    # Add model type column
    df_return["model"] = "Daily Returns"
    df_return_vol["model"] = "Volume and Daily Returns"

    # Combine the data frames
    df_combined = pd.concat([df_return, df_return_vol], ignore_index=True)

    # Create the plot
    fig, axes = plt.subplots(nrows=4, ncols=3, figsize=(18, 12))
    axes = axes.flatten()  # Flatten axes array for easy iteration

    # Loop through each stock
    for i, stock in enumerate(stocks):
        # Filter data for the current stock
        df_filtered = df_combined[df_combined["Stock"] == stock]

        # Create boxplot
        sns.boxplot(x="model", y="CWC", hue="model", palette="mako", data=df_filtered, ax=axes[i], whis=(0, 100))
        axes[i].set_title(stock)
        axes[i].set_xlabel("Features")
        axes[i].set_ylabel("CWC")

    # Remove unused axes
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    # Adjust layout and display plot
    plt.tight_layout()
    plt.show()

In [ ]:
# Retrieving the final results of the comparison between QLSTM and GARCH:

results_comp = pd.read_csv("Final_comparison_results/final_results_QLSTM.csv")
results_comp_2 = pd.read_csv("Final_comparison_results/final_results_GARCH.csv")

for period in sorted(results_comp["Period"].unique()):
    period_df = results_comp[results_comp["Period"] == period]
    period_df_2 = results_comp_2[results_comp_2["Period"] == period]
    
    stats_df = period_df.groupby("Stock").agg(
        Best=("CWC", "max"),
        Mean=("CWC", "mean"),
        std=("CWC", "std")
    ).reset_index()
    
    stats_df_garch = period_df_2.groupby("Stock").agg(
    Result=("CWC", "max"),
    ).reset_index()

    print(f"Results for period {period}:\n", stats_df, "\n")
    print(stats_df_garch)



In [ ]:
# Plotting the PICP and PINAW for the QLSTM and GARCH:

results_comp = pd.read_csv("Final_comparison_results/final_results_QLSTM.csv")
results_comp_2 = pd.read_csv("Final_comparison_results/final_results_GARCH.csv")

# Add a column to identify the model
results_comp["Model"] = "QLSTM"
results_comp_2["Model"] = "GARCH"

# Concatenate the two datasets
combined_results = pd.concat([results_comp, results_comp_2])

# Reset the index to avoid duplicate index issues
combined_results.reset_index(drop=True, inplace=True)

# Get unique periods from the dataset
unique_periods = combined_results["Period"].unique()

# Loop through all periods
for period in unique_periods:
    # Filter data for the current period
    period_data = combined_results[combined_results["Period"] == period]

    # Plotting "PINAW" vs "PICP" in a 2D scatter plot for the current period
    plt.figure(figsize=(12, 6))
    sns.scatterplot(x="PINAW", y="PICP", hue="Model", style="Stock", data=period_data, palette="Set2")

    # Add a horizontal line at y = 0.9
    plt.axhline(y=0.9, color="grey", linestyle="--", linewidth=1)

    # Adjust legend to only show model names
    handles, labels = plt.gca().get_legend_handles_labels()
    model_handles = [handle for handle, label in zip(handles, labels) if label in ["QLSTM", "GARCH"]]
    model_labels = [label for label in labels if label in ["QLSTM", "GARCH"]]
    plt.legend(model_handles, model_labels, title="Model", loc="upper left", bbox_to_anchor=(0, 1))

    # Set the labels
    plt.xlabel("PINAW")
    plt.ylabel("PICP")

    # Show the plot
    plt.show()

In [ ]:
# Creating a legend for the stocks:     

# Read the data
results_comp = pd.read_csv("Final_comparison_results/final_results_QLSTM.csv")
results_comp_2 = pd.read_csv("Final_comparison_results/final_results_GARCH.csv")

# Combine the datasets
combined_results = pd.concat([results_comp, results_comp_2])

# Plot just to get the handles and labels (no need to show the plot)
scatter = sns.scatterplot(x="PINAW", y="PICP", style="Stock", data=combined_results, color = "black")

# Extract the handles and labels for the stocks only (ignore QLSTM and GARCH)
handles, labels = scatter.get_legend_handles_labels()
stock_handles = [handle for handle, label in zip(handles, labels) if label not in ["QLSTM", "GARCH"]]
stock_labels = [label for label in labels if label not in ["QLSTM", "GARCH"]]

# Create and display the stock legend separately
fig_legend = plt.figure(figsize=(0.5, 6))
fig_legend.legend(stock_handles, stock_labels, title="Stock", loc="center", frameon=True)
    
plt.axis('off')
plt.show()